### Install detectron2

We are using detectron2 library for training and evaluation. Detectron2 consists of mask-rcnn algorithms for image segmentation and various pre-trained models which can be used to train the models by tweaking some of the parameters. Detectron betters works on GPU and need to install pytorch to run on the machine. It is recommeded to install Cuda, Cudnn, pytorch and opencv for better training and inference.

In [ ]:
!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

In [ ]:
# Determining the path required for dataset/

class Paths:
    DATASET_DIR = "dataset"
    TRAIN_IMAGES_DIR = f"{DATASET_DIR}/train/images"
    TRAIN_ANNOTATIONS = f"{DATASET_DIR}/train/annotations.json"
    VALID_IMAGES_DIR = f"{DATASET_DIR}/val/images"
    VALID_ANNOTATIONS = f"{DATASET_DIR}/val/annotations.json"


class DatasetLabels:
    TRAIN = "dataset_train"
    VAL = "dataset_val"

In [ ]:
import os
from multiprocessing import Pool
from collections import Counter
import json
import matplotlib.pyplot as plt
import shutil
from loguru import logger

from tqdm.notebook import tqdm
from pycocotools.coco import COCO
import numpy as np
import cv2

from detectron2.data.datasets import register_coco_instances, register_coco_panoptic
import detectron2
from detectron2.utils.logger import setup_logger
from detectron2.utils.visualizer import Visualizer
from detectron2.utils.visualizer import ColorMode
from detectron2.data import MetadataCatalog
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
from detectron2.engine import DefaultPredictor
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg
from detectron2 import model_zoo
from detectron2.utils.events import get_event_storage
from detectron2.engine import HookBase

### Visualize the dataset
It is extremely important to visualize the dataset before feeding it into the model. I have used COCOEvalutor to visualize the data. All the annotations are present in COCO format. 

In [ ]:
from pycocotools.coco import COCO
import numpy as np
import skimage.io as io

In [ ]:
coco = COCO(Paths.VALID_ANNOTATIONS)
catIDs = coco.getCatIds()
cats = coco.loadCats(catIDs)

In [ ]:
labels = [""]
catIds = coco.getCatIds(catNms=labels)
imgIds = coco.getImgIds(catIds)
print("Total images:",len(imgIds))
print(imgIds[np.random.randint(0,len(imgIds))])
imgs = coco.loadImgs(imgIds[np.random.randint(0,len(imgIds))])[0]
img_filename = imgs.get("file_name")
I = io.imread(os.path.join(Paths.VALID_IMAGES_DIR,img_filename))/255
plt.imshow(I)
plt.show()

In [ ]:
# Load and display instance annotations
plt.imshow(I)
plt.axis('off')
annIds = coco.getAnnIds(imgIds=imgs['id'], catIds=catIds, iscrowd=None)
anns = coco.loadAnns(annIds)
coco.showAnns(anns)

In [ ]:
with open(Paths.TRAIN_ANNOTATIONS) as fp:
    dest_train_annotations = json.load(fp)
with open(Paths.VALID_ANNOTATIONS) as fp:
    dest_valid_annotations = json.load(fp)

In [ ]:
# Detectron2 requires the dataset to be registered in their instances before feeding into the network

_ = setup_logger()

register_coco_instances(DatasetLabels.TRAIN, {}, Paths.TRAIN_ANNOTATIONS, Paths.TRAIN_IMAGES_DIR)
register_coco_instances(DatasetLabels.VAL, {}, Paths.VALID_ANNOTATIONS, Paths.VALID_IMAGES_DIR)

In [ ]:
# Building the configuration for training

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_101_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = (DatasetLabels.TRAIN,)
cfg.DATASETS.TEST = ()
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 8 # Number of output classes

cfg.OUTPUT_DIR = "outputs"
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_101_FPN_3x.yaml")
cfg.SOLVER.IMS_PER_BATCH = 2

In [ ]:
cfg.SOLVER.BASE_LR = 0.00025  # Learning Rate
cfg.SOLVER.MAX_ITER = 30000  # MAx Iterations
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128  # Batch Size

In [ ]:
# Start training
trainer = DefaultTrainer(cfg) 
trainer.resume_or_load(resume=False)
trainer.train()

In [ ]:
# Setting up trained models for evaluation

cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.DATASETS.TEST = (DatasetLabels.VAL, )
predictor = DefaultPredictor(cfg)

In [ ]:
# Evaluating the trained model quantitatively

evaluator = COCOEvaluator(DatasetLabels.VAL, cfg, False, output_dir=cfg.OUTPUT_DIR)
data_loader = build_detection_test_loader(cfg, DatasetLabels.VAL)
results = inference_on_dataset(predictor.model, data_loader, evaluator)

In [ ]:
# Evaluating the model quantitatively

test_img_path = "008220.jpg"  # Need to add the path for images
test_img = cv2.imread(test_img_path)
predictions = predictor(test_img)
v = Visualizer(test_img[:, :, ::-1],
    scale=0.5, 
      )
annotated_image = v.draw_instance_predictions(predictions["instances"].to("cpu"))
plt.imshow(annotated_image.get_image())